# 10.3 - First Order Logic
In this chapter, we will express language expressions by translating them to first-order logic.
## Syntax
First-Order Logic keeps all the boolean operators of Propositional Logic but it adds some important new mechanisms. 
* Propositions are analysed into predicates and arguments. 
* The standard construction rules for first-order logic recognize **terms** as individual variables and individual constants, and **predicates**, a function that tells us what the subject does or is, which take differing numbers of arguments.

A **predicate** is an expression that can be *true* of something. For example, the expression `is moving` is true of anything that is moving. Some examples of predicates are:

* She **dances** (verb-only predicate)
* Ben **reads the book** (verb-plus-direct-object predicate) denoted as `read(book)`
* She **gave me a present** (verb-plus-indirect-object-plus-direct-object predicate) denoted as `give(me, a present)`
* They **elected her president** (verb-plus-indirect-object-plus-predicative-noun predicate) denoted as `elect(her, president)`

For example, 
* `Angus walks` is formalized as a a **unary predicate** `walks(angus)`.
* `Angus sees Bertie` is formalized as a **binary predicate** `see(angus, bertie)`.  

The symbols used in predicates do not have intrinsic meaning. There is *no logical difference* between the following:

1. `love(margaret, lucy)` (in English)
2. `sarang(margaret, lucy)` (in Korean)

First-order logic has nothing substantiative to say about lexical semantics (the meaning of individual words). Whether an atomaic predication like `see(angus, beetle)` is true or false is not a matter of logic, but depends on the particular valuation of the *constants* we have chosen, in this case, `see`, `angus` and `beetle`. For this reason, such expressions are called **non-logical constants**. By contrast, **logical constants** like boolean operators always receive the same interpretation in every model of first-order logic.

A binary predicate that has special status to note is equality, in formulas like `angus = aj`. Equality is regarded as a logical constant. Because for 2 entities, $t_1$ and $t_2$, the formula $t_1=t_2$ is only trueif and only if $t_1$ and $t_2$ refer to one and the same entity.

To inspect the syntactic structure of expressions of first-order logic, we often assign **types** to expressions. We use two **basic types**, $e$ is the **type of entities** while $t$ is the **type of formulas** i.e. expressions which have truth values. 

Given these two basic types, $e$ and $t$, we can form **complex types** for function expressions. That is, given any types $\sigma$ and $\tau$, $\left<\sigma, \tau\right>$ is a complex type corresponding to functions from "$\sigma$ things" to "$\tau$ things". For example, $\left<e, t\right>$ is the type of expressions from entites to truth values (see above paragraph), namely unary predicates. The logical expression can be processed with type checking.

In [1]:
import nltk
read_expr = nltk.sem.Expression.fromstring
expr = read_expr('walk(angus)', type_check=True)
expr.argument

<ConstantExpression angus>

In [2]:
expr.argument.type

e

In [3]:
expr.function

<ConstantExpression walk>

In [4]:
#Notice that it's a ? here
expr.function.type

<e,?>

Notice the final result `<e, ?>`. Although the type-checker will try to infer as many times as possible, in this case it has not managed to fully specify the type. $t$ of `walk`, since its result type is unknown. We intend `walk` to have the type `<e, t>` as far as the type-checker knows, in this context it could be of some other type such as `<e, e>` or `<e, <e, t>`.

To help the type-checker, we need to specify a **signature**, implemented as a dictionary that associates types with non-logical constants:

In [5]:
sig = {'walk' : '<e, t>'}
expr = read_expr('walk(angus)', signature=sig)
print(expr.function)
print(expr.function.type)

walk
e


A binary predicate has type $\left<e, \left<e,t\right>\right>$. Although this is the type which combines first with an argument of type $e$ to make a unary predicate, we represent binary predicates as directly combining directly with their two arguments. For example, in the sentence `Angus sees Cyril`, we explicitly identify the unary predicate `see(cyril)`. The predicate `see` in the translation of the unary predicate will combine with its arguments to give the result `see(angus, cyril)`.

In first-order logic, the arguments can also be individual variables like $x$, $y$ and $z$. Individual varriables are similar to personal pronouns `he`, `she` and `it` and we need to know the context to figure out their denotation. One way of interpreting the pronoun in the sentence `He disappeared` is by pointing to a relevant individual in the local context. (Explicitly, what proper noun does `He` in `He disappeared` refer to in some passage?)

Let's say the passage is `Cyril is Angus's dog. He disappeared.` Here, we say `He` is **coreferential** with the noun phrase `Cyril`. As a result, the above sentence is sementically equivalent to `Cyril is Angus's dog. Cyril disappeared.`

Contrast this with the occurence of `he` in the following sentence: `Angus had a dog but he disappeared`. This sentence is not sementically equivalent to `Angus had a dog but a dog disappeared`. 

We can construct an **open formula** with two occurences of the variable $x$:

* He is a dog and he disappeared
* $\text{dog}(x) \wedge \text{disappear}(x)$

Let's continue. by placing an **existential quantifier, $\exists x$** (translated to 'for some $x$') in front of the above open formula, we can bind these variables:

* $\exists x.(dog(x) \wedge disappear(x))$ (for some entity)
* At least one entity is a dog and the dog disappeared.
* A dog disappeared.

The NLTK rendering of the above expression is:

`exists x.(dog(x) & disappear(x))`

In addition to the existential quantifier, first-order logic offers us the **universal quantifier, $\forall x$** (translated to 'for all '$x$') illustrated as follows:

* $\forall x .(dog(x) \rightarrow disappear(x))$
* Everything has the property that if it is a dog, it disappears.
* Every dog disappeared

The NLTK syntax for the above expression is:

`all x.(dog(x) -> disappear(x))`

Note that the above expression *does not state if there are any dogs*. So in a situation where there are no dogs, `all x (dog(x) -> disappear(x))` will still come out true.

Now what happens in formulas like the following:

`((exists x.dog(x)) -> bark(x))`

The scope of the `exists x` quantifier is `dog(x)`, so the occurence of `x` in `bark(x)` is **unbound**. Consequently, it can become bound by some other quantifier, for example `all x` in the next formula:

`all x.((exists x.dog(x)) -> bark(x))`

In general, an occurence of a variable $x$ in a formula $\phi$ is **free** in $\phi$ if that occurence does not fall within the scope of `all x` or `some x` in $\phi$. Conversely, if $x$ is free in formula $\phi$, then it is **bound** in `all x.`$\phi$ and `exists x.`$\phi$. If all variable occureneces in a formula are bound, the formula is said to be **closed**.

We mentioned before that the `Expression` object can process strings and returns objects of class `Expression`. Each instance `expr` of this class comes with a method `free()` which returns the set of variables that are free in `expr`.

In [6]:
read_expr = nltk.sem.Expression.fromstring
read_expr('dog(cyril)').free()

set()

In [7]:
read_expr('dog(x)').free()

{Variable('x')}

In [8]:
read_expr('own(angus, cyril)').free()

set()

In [9]:
read_expr('exists x.dog(x)').free()

set()

In [10]:
read_expr('((some x.walk(x)) -> sing(x))').free()

{Variable('x')}

In [11]:
read_expr('exists x.own(y, x)').free()

{Variable('y')}

## First-Order Theorem Proving

Recall the constraint `to the north of` in 10.2. Prepositional logic is not expressive enough to represent generalizations about binary predicates. But we can formalize it in first-order logic:

* $\forall x \forall y.(north\_of(x, y) \rightarrow -north\_of(y, x))$

`all x. all y.(north_of(x, y) -> -north_of(y, x)`

Even better, we can perform automated inference to show the validity of the argument.

The general case in theorem proving is to determine whether a formula that we want to prove (a **proof goal**) can be derived by a finite sequence of inference steps from a list of assumed formulas. We write this as $S \vdash g$ where 
* $S$ is a (possibly empty) list of assumptions and
* $g$ is a proof goal

## Summarizing the Language of First Order Logic
Let's restate our earlier syntactic rules for propositional logic and add the formation rules for quantifers to give us the syntax of first-order logic.

We'll adopt the convention that $\left<e^n, t\right>$ and $\alpha_1, \cdots, \alpha_n$ is th type of a predicate which combines $n$ arguments of type $e$ to yield an expression of type $t$. We say that $n$ is the **arity** (no. of arguments) of the predicate. Now...

1. If $P$ is a predicate of type terms of type $\left<e^n, t\right>$ and $\alpha_1, \cdots, \alpha_n$ are terms of type $e$, then $P(\alpha_1, \cdots, \alpha_n$) is of type $t$
2. If $\alpha$ and $\beta$ are both of type $e$, then $(\alpha = \beta)$ and $(\alpha$ `!=` $\beta)$ are of type $t$.
3. If $\phi$ is of type $t$, then so is $-\phi$
4. If $\phi$ and $\psi$ are of type $t$, then so are $(\phi \text{  } \& \text{  } \psi)$, $(\phi \text{  } | \text{  } \psi)$, $(\phi \rightarrow \psi)$ and $(\phi \leftrightarrow \psi)$
5. If $\phi$ is of type $t$ and $x$ is a variable of type $e$, then `exists x.`$\phi$ and `all x.`$\phi$ are of type $t$

The following table summarizes the new logical constants of the `logic` module, and two of the methods of `Expression`s.

| Example | Description   |
|------|------|
| `=` | equality|
| `!=` | inequality|
| `exists` | existential quantifier |
| `all` | universal quantifier |
| `e.free()` | show free variables of `e` |
| `e.simplify()` | carry out $\beta$-reduction on `e` |

## Truth in Model
To get us further forward if we can give meaning to sentences in first-order logic, we need to give a *truth-conditional semantics* to first-order logic. From the point of view of computational semantics, there are obvious limits in how far one can push this approach. However, it is still possible to gain a clearer picture of truth-conditional semantics by endocing models in NLTK.

Given a first-order language $L$, a model $M$ for $L$ is a pair $\left<D, Val\right>$, where $D$ is a non-empty set called the **domain** of the model and $Val$ is the **valuation function** which assigns values from $D$ to expressions of $L$ adhering to the following rules:

1. For every individual constant $c$ in $L$, $Val(c)$ is an element of $D$.
2. For every predicate symbol $P$ of arity $n \geq 0$, $Val(P)$ is a function from $D^n$ to ${True, False}$. If the arity of $P$ is $0$, then $Val(P)$ is simply a truth value and $P$ is regarded as a propositional symbol.

For example, if $P$ is of arity $2$ then $Val(P)$ will be a function $f$ from *pairs of elements* of $D$ to ${True, False}$. In the models we shall build in NLTK, we'll adopt the more convenient alternative, in whic $Val(P)$ is a set $S$ of pairs, defined as follows:

$$S = \left\{s \text{  } | \text{  } f(s) = True\right\}$$

Such $f$ is called the **characteristic function** of $S$.

Relations are represented semantically in NLTK in the standard set-theoretic way, as sets of tuples. For example, let's suppose we have a domain of discourse consisting of the individuals Bertie, Olive and Cyril, where Bertie is a boy, Olive is a girl and Cyril is a dog. Let's use the labels `b`, `o` and `c` as the corresponding labels.

In [12]:
dom = {'b', 'o', 'c'}

In [13]:
v = """
bertie => b
olive => o
cyril => c
boy => {b}
girl => {o}
dog => {c}
walk => {o, c}
see => {(b, o), (c, b), (o, c)}
"""
val = nltk.Valuation.fromstring(v)
val

{'bertie': 'b',
 'boy': {('b',)},
 'cyril': 'c',
 'dog': {('c',)},
 'girl': {('o',)},
 'olive': 'o',
 'see': {('b', 'o'), ('c', 'b'), ('o', 'c')},
 'walk': {('c',), ('o',)}}

Notice there are also unary predicates (`boy`, `girl`, `dog` as singleton tuples. This is a convenience which allows us to have a uniform treatment of relations of any arity..

In [14]:
('o', 'c') in val['see']

True

In [15]:
('b',) in val['boy']

True

## Individual Variables and Assignments

In our models, the counterpet of a context of use is a variable **assignment**. This is a mapping from individual variables to entities in the domain. Assignments are created using the `Assignment` constructor, which also takes the model's domain of discourse as a parameter.

In [16]:
g = nltk.Assignment(dom, [('x', 'o'), ('y', 'c')])
g

{'x': 'o', 'y': 'c'}

In [17]:
#print() format that gives a notation closer to those found in logic textbooks
print(g)

g[c/y][o/x]


Now we can evaluate an atomic formula of first-order logic. We create a model then call the `evaluate()` method to compute the truth value.

We are evaluating a formu;a which is similar to our earlier example, `see(olive, cyril)`. 

In [18]:
m = nltk.Model(dom, val)
m.evaluate('see(olive, y)', g)

True

However when the interpretation function encounters the variable `y`, rather than checking for a value in `val`, it asks the variable assignment `g` to come up with a value:

In [19]:
g['y']

'c'

Since we already know that the individuals `o` and `c` stand in the `see` relation, the value `True` is as expected. We say that the assignment `g` **satisfies** the formula `see(olive, y)`. In contrast, the following formula evaluates to `False`.

In [20]:
m.evaluate('see(y, x)', g)

False

In [21]:
#Clear all bindings from an assignment
g

{'x': 'o', 'y': 'c'}

In [22]:
g.purge()
g

{}

Now when we try to evaluate a formula `see(olive, y)` relative to `g`, it's like trying to interpret a sentence containing a *him* in the sentence but we do not know what *him* refers to. In this case, the evaluation fails to deliver a truth value.

In [23]:
m.evaluate('see(olive, y)', g)

'Undefined'

Since our models already contain rules for interpreting boolean operators, arbitrarily complex formulas can be composed and evaluated.

In [24]:
m.evaluate('see(bertie, olive) & boy(bertie) & -walk(bertie)', g)

True

This process is called **model checking**.

## Quantification

One of the crucial insights of modern logic is that the notion of variable satisfaction can be used to provide an interpretation to quantified formulas. Let's use an example

`exists x.(girl(x) & walk(x))`

For this we will want to check if there are any individuals in our domain, `dom`, that have the property of being a girl and walking. In other words, we want to find if there is some $u$ in `dom` such that `g[u/x]` satisfies the open formula:

`(girl(x) & walk(x))`

Consider the following:

In [25]:
m.evaluate('exists x.(girl(x) & walk(x))', g)

True

This returns `True` because there is some $u$ in `dom` such that `(girl(x) & walk(x))` is satisfied by an assignment that binds `x` to $u$. In fact `o` is such a $u$:

In [26]:
m.evaluate('exists x.(girl(x) & walk(x))', g.add('x', 'o'))

True

One useful tool offered by NLTK is the `satisfiers()` method. This returns a set of all the individuals that satisfy an open formula. The method parameters are a parsed formua, a variable and an assignment. Here are some examples:

In [27]:
fm1a1 = read_expr('girl(x) | boy(x)')
m.satisfiers(fm1a1, 'x', g)

{'b', 'o'}

In [28]:
fm1a2 = read_expr('girl(x) -> walk(x)')
m.satisfiers(fm1a2, 'x', g)

{'b', 'c', 'o'}

In [29]:
fm1a3 = read_expr('walk(x) -> girl(x)')
m.satisfiers(fm1a3, 'x', g)

{'b', 'o'}

In [30]:
m.evaluate('all x.(girl(x) -> walk(x))', g)

True

In other words, a universally quantified formula $\forall x.\phi$ is true with respect to `g` just in case for every $u$, $\phi$ is true with respect to `g[u/x]`.

## Quantifier Scope Ambiguity

What happens when we want to give a formal representation of a sentence with `two` quantifiers, such as the following?

* Everybody admires someone

There are (at least) two ways of expressing the above in first-order logic:

1. `all x.(person(x) -> exists y.(person(y) & admire 
(x,y)))`
2. `all x.(person(x) -> exists y.(person(y) -> admire (x,y)))`

Both can be used, but they have different meanings. (2) is logically stronger than (1).

(2) claims that there is a unique person, say Bruce, who is admired by everyone. (1) just requires that for every person $u$, we can find some person $u'$ whom $u$ admires, but it could be a different $u'$ in each case. We distinguish (1) and (2) in terms of the **scope** of the quantifiers. In (1), $\forall$ has a wider scope then $\exists$, while in (2), the ordering is reversed.

In order to examine the ambiguity more closely, let's fix the valuation as follows:

In [31]:
>>> v2 = """
... bruce => b
... elspeth => e
... julia => j
... matthew => m
... person => {b, e, j, m}
... admire => {(j, b), (b, b), (m, e), (e, m)}
... """
>>> val2 = nltk.Valuation.fromstring(v2)
val2

{'admire': {('b', 'b'), ('e', 'm'), ('j', 'b'), ('m', 'e')},
 'bruce': 'b',
 'elspeth': 'e',
 'julia': 'j',
 'matthew': 'm',
 'person': {('b',), ('e',), ('j',), ('m',)}}

One way for exploring the results is using the `satisfiers()` method:

In [32]:
dom2 = val2.domain
m2 = nltk.Model(dom2, val2)
g2 = nltk.Assignment(dom2)
fmla4 = read_expr('person(x) -> exists y.(person(y) & admire(x, y))')
m2.satisfiers(fmla4, 'x', g2)

{'b', 'e', 'j', 'm'}

This shows that `fmla4` holds for every individual in the domain. By contrast, consider the formula `fmla5` below. There are no satisfiers for the variable `y`:

In [33]:
fmla5 = read_expr('(person(y) & all x.(person(x) -> admire(x, y)))')
m2.satisfiers(fmla5, 'y', g2)

set()

This means there is no person that is admired by everybody. Let's use a different formula:

In [34]:
fmla6 = read_expr('(person(y) & all x.((x = bruce | x = julia) -> admire(x, y)))')
m2.satisfiers(fmla6, 'y', g2)

{'b'}